In [15]:
# Establish connection to database and create cursor
connection = psycopg2.connect(user = "postgres",
                              password = pg_pw,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "PH-EmployeeDB")

cursor = connection.cursor()

In [30]:
# Get dept_name headers
columns = []
sales_header_query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'retiring_titles'"
cursor.execute(sales_header_query)
query_results = cursor.fetchall()
print(query_results)


[('emp_no',), ('first_name',), ('last_name',), ('dept_name',)]


In [39]:
# Define function to get table header names

#!/usr/bin/python3
# -*- coding: utf-8 -*-

# import the sql and connect libraries for psycopg2
from psycopg2 import sql, connect

# create a global string for the PostgreSQL db name
db_name = "PH-EmployeeDB"

try:
    # declare a new PostgreSQL connection object
    conn = connect(
        dbname = db_name,
        user = "postgres",
        host = "localhost",
        password = pg_pw
    )

    # print the connection if successful
    print ("psycopg2 connection:", conn)

except Exception as err:
    print ("psycopg2 connect() ERROR:", err)
    conn = None

# define a function that gets the column names from a PostgreSQL table
def get_columns_names(table):

    # declare an empty list for the column names
    columns = []

    # declare cursor objects from the connection    
    col_cursor = conn.cursor()

    # concatenate string for query to get column names
    # SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'some_table';
    col_names_str = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE "
    col_names_str += "table_name = '{}';".format( table )

    # print the SQL string
    #print ("\ncol_names_str:", col_names_str)

    try:
        sql_object = sql.SQL(
            # pass SQL statement to sql.SQL() method
            col_names_str
        ).format(
            # pass the identifier to the Identifier() method
            sql.Identifier( table )
        )

        # execute the SQL string to get list with col names in a tuple
        col_cursor.execute( sql_object )

        # get the tuple element from the liast
        col_names = ( col_cursor.fetchall() )

        # print list of tuples with column names
        #print ("\ncol_names:", col_names)

        # iterate list of tuples and grab first element
        for tup in col_names:

            # append the col name string to the list
            columns += [ tup[0] ]

        # close the cursor object to prevent memory leaks
        col_cursor.close()

    except Exception as err:
        print ("get_columns_names ERROR:", err)

    # return the list of column names
    return columns

# if the connection to PostgreSQL is valid
if conn != None:

    # pass a PostgreSQL string for the table name to the function
    columns = get_columns_names( "dept_info" )

    print ("columns:", columns)
    print ("columns TYPE:", type(columns))


psycopg2 connection: <connection object at 0x11b1459f0; dsn: 'user=postgres password=xxx dbname=PH-EmployeeDB host=localhost', closed: 0>
columns: ['emp_no', 'first_name', 'last_name', 'dept_name']
columns TYPE: <class 'list'>


In [38]:
# Get sales department info
sales_dept_query = "SELECT * FROM dept_info WHERE dept_name = 'Sales'"
cursor.execute(sales_dept_query)
query_results = cursor.fetchall()
sales_dept_df = pd.DataFrame(query_results)
# Add headers to DataFram
columns = get_columns_names( "dept_info" )
sales_dept_df.columns = columns
sales_dept_df

,emp_no,first_name,last_name,dept_name
0,10053,Sanjiv,Zschoche,Sales
1,10088,Jungsoon,Syrzycki,Sales
2,10215,Xiaobin,Duclos,Sales
3,10355,Satyanarayana,Cochrane,Sales
4,10463,Ung,Zaiane,Sales
...,...,...,...,...
5855,499689,Kinh,Sommer,Sales
5856,499817,Lihong,Pusterhofer,Sales
5857,499920,Christ,Murtagh,Sales
5858,499966,Mihalis,Crabtree,Sales
